In [1]:
from langchain.prompts import ChatPromptTemplate 
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import YoutubeLoader
from langchain_community.document_loaders import WebBaseLoader
from youtube_transcript_api import YouTubeTranscriptApi
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, Runnable
import bs4
from operator import itemgetter
import markdown
import datetime
import pytube
import os

async def web_loader(url: str):
    loader = WebBaseLoader(web_paths=[url])
    docs = []
    async for doc in loader.alazy_load():
        docs.append(doc)
    return docs

def chain_summarization(content: str, size: str = 'medium'):
    
    
    if size == 'small':
        messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar textos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de uma página web e gerar uma sumarização curta que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), ('user', 'aqui está o conteudo {texto}')] 
    elif size == 'medium':
        messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar textos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de uma página web e gerar uma sumarização de tamanho médio que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), ('user', 'aqui está o conteudo {texto}')]
    elif size == 'large':
        messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar textos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de uma página web e gerar uma sumarização longa e detalhada que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), ('user', 'aqui está o conteudo {texto}.')]

    prompt = ChatPromptTemplate.from_messages(messages)
    llm = ChatOpenAI(model='gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))
    chain = (
        {
            'texto': itemgetter('texto')
        }
        |prompt
        | llm
        |StrOutputParser()
    )

    resume = chain.invoke({'texto': content})
    return resume

def youtube_summarization(url: str, size: str = 'medium'):
    
    url = url.strip()
    loader = YoutubeLoader.from_youtube_url(url, language=['pt', 'en'], translation='pt')
    texto = loader.load()
    pytube_loader = pytube.YouTube(url)
    title = pytube_loader.title
    
    if size == 'small':
        messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar vídeos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de um vídeo e gerar uma sumarização curta que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), ('user', 'aqui está o conteudo {texto}')] 
    elif size == 'medium':
        messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar vídeos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de um vídeo e gerar uma sumarização de tamanho médio que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), ('user', 'aqui está o conteudo {texto}')]
    elif size == 'large':
        messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar vídeos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de um vídeo e gerar uma sumarização longa e detalhada que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), ('user', 'aqui está o conteudo {texto}.')]

    prompt = ChatPromptTemplate.from_messages(messages)
    llm = ChatOpenAI(model = 'gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))
    chain = (
        {
            'texto': itemgetter('texto')
        }
        |prompt
        | llm
        |StrOutputParser()
    )

    resume = chain.invoke({'texto': texto})
    return resume, title

def chain_youtube_summarization(url: str, size: str) -> Runnable:
    """ 
    Responsável por criar uma chain que irá sumarizar um vídeo do YouTube.

    Args:
        url (str): A URL do vídeo do YouTube.
        size (str): O tamanho de parágrafos para o resumo.

    Returns:
        Runnable: Um Runnable que executa a sumarização do vídeo do YouTube.

    Raises:
        HTTPException: Se houver um erro ao acessar o vídeo do YouTube.
    """
    
    try:
        video_id = url.find('v=')
        video_id = url[video_id+2:]
        transcript = YouTubeTranscriptApi.get_transcript(video_id)

        texto = ""
        for line in transcript:
            texto += line['text'] + " "
    except Exception as e:
        return 'conteudo indisponivel'


    messages = [('system', 'Você é um assistente virtual com o objetivo de sumarizar vídeos de maneira clara, didática e objetiva. Sua tarefa é receber o conteúdo de um vídeo e gerar uma sumarização curta que destaque os principais pontos, explicando-os de forma acessível. A ênfase deve ser na simplicidade, garantindo que o usuário compreenda todos os conceitos e informações chave. Ao fazer a sumarização, priorize a clareza, organização e a explicação de cada ponto central do conteúdo enviado.'), 
    ('system', f'Utilize até, no MÁXIMO, {size} parágrafos para o resumo.'),
    ('user', f'aqui está o conteudo {texto}')] 

    
    prompt = ChatPromptTemplate.from_messages(messages)
    llm = ChatOpenAI(model = 'gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'))
    chain = (
        prompt
        | llm
        | StrOutputParser()
    )

    return chain.invoke({})

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
content = await web_loader('https://www.uol.com.br')
trim_content = "\n".join([x.page_content for x in content])[:2_000]
summary = chain_summarization(trim_content, 'medium')
summary

'O conteúdo fornecido parece ser uma lista extensa de serviços, produtos e seções disponíveis no site UOL (Universo Online). Aqui está um resumo dos principais pontos:\n\n1. **Serviços e Produtos**: O UOL oferece uma variedade de produtos e serviços, incluindo hospedagem (UOL Host), pagamentos (PagBank), e plataformas de entretenimento (UOL Play). Isso sugere que o UOL não é apenas um portal de notícias, mas também um provedor de soluções digitais.\n\n2. **Assinatura**: Há uma opção de assinatura que permite acesso ilimitado ao conteúdo exclusivo do UOL, indicando que o portal valoriza a oferta de informações premium para seus assinantes.\n\n3. **Canal de Comunicação**: O UOL disponibiliza suporte ao cliente por telefone, com números específicos para capitais e outras localidades, facilitando o contato para dúvidas ou assistência.\n\n4. **Variedade de Conteúdos**: O site abrange uma gama diversificada de tópicos, como notícias, política, economia, educação e entretenimento, além de seç

In [2]:
content_yt = chain_youtube_summarization('https://www.youtube.com/watch?v=5p248yoa3oE&t=416s', 4)

In [3]:
content_yt

'No vídeo, Dr. Andrew Ng, um renomado especialista em inteligência artificial (IA), discute as oportunidades e o futuro da tecnologia de IA, destacando sua natureza como uma "tecnologia de propósito geral", semelhante à eletricidade, que pode ser aplicada em diversas áreas. Ele enfatiza a importância de duas ferramentas principais de IA: aprendizado supervisionado e IA generativa, explicando que o aprendizado supervisionado é eficaz para rotular e classificar dados, como identificar e-mails de spam, enquanto a IA generativa, como os modelos de linguagem, pode criar texto e conteúdo a partir de prompts.\n\nNg compartilha experiências de projetos bem-sucedidos, como otimização de rotas de navios e sistemas de monitoramento de avaliações de restaurantes, demonstrando como o aprendizado supervisionado pode gerar valor significativo. Ele observa que, enquanto o aprendizado supervisionado já é uma área valiosa, a IA generativa está começando a expandir suas aplicações, permitindo que desenvo

In [ ]:
import discord
from discord.ext import commands

import nest_asyncio
import os
nest_asyncio.apply()
from dotenv import load_dotenv

load_dotenv()

TOKEN = os.getenv('DISCORD_TOKEN')

intents = discord.Intents.all()
intents.message_content = True

bot = commands.Bot(command_prefix='$', intents=intents)

def get_metadata_from_message(message):
    return {
        'message_id': message.id,
        'channel': message.channel,
        'author': message.author,
        'content': message.content,
    }
    
@commands.command()
async def get_summary(ctx, *args):
    arguments = ', '.join(args)
    
    
    await ctx.send(f'Processando o link...')
    type_content = args[0]
    url = args[1]
    
    await ctx.send(f'Type content: {args[0]}')
    await ctx.send(f'URL: {args[1]}')
    
    try:
        if type_content.lower() == 'web':
            content = await web_loader(url)
            trim_content = "\n".join([x.page_content for x in content])[:2_000]
            summary = chain_summarization(trim_content, 'medium')
            await ctx.send(summary)
            
        elif type_content.lower() == 'youtube':
            content_yt = chain_youtube_summarization(url, 4)
            await ctx.send(content_yt)
        
    except Exception as e:
        print(e)
        await ctx.send(f'Erro ao acessar a página {url}')
        return
    
bot.add_command(get_summary)
bot.run(TOKEN)

[2024-11-18 14:01:47] [INFO    ] discord.client: logging in using static token
[2024-11-18 14:01:49] [INFO    ] discord.gateway: Shard ID None has connected to Gateway (Session ID: 01fbc0981b8f0ea6fd7b9779204d7e67).
